In [1]:
#imports
import os
import operator
from dotenv import load_dotenv, find_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_huggingface import ChatHuggingFace
from langchain_huggingface import HuggingFaceEndpoint
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver
from templates.templates import templates
from typing import Annotated, TypedDict

In [2]:
#load env variables

_ = load_dotenv(find_dotenv())

In [3]:
#Define search tool

searchTool = TavilySearchResults(max_results=2)

In [4]:
#Define agent state
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [5]:
#Define memory for conversation history
memory = SqliteSaver.from_conn_string(":memory:")

In [6]:
#Generate chat model from HuggingFace and Llama 3 70B

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm = HuggingFaceEndpoint(
    repo_id=model_id,
    task="text-generation",
    temperature=0.2,
    max_new_tokens=512,
    do_sample=False,
)

chat_model = ChatHuggingFace(llm=llm)

/home/pingu/anaconda3/envs/test-hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/pingu/.cache/huggingface/token
Login successful


In [7]:
#Define agent
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_llm)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {tool.name: tool for tool in tools}
        self.model = model.bind_tools(tools)

    def call_llm(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for tool in tool_calls:
            print(f"Calling: {tool}")
            result = self.tools[tool['name']].invoke(tool['args'])
            results.append(ToolMessage(tool_call_id=tool['id'], name=tool['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [24]:
prompt = """
For the following message extract the following information:

task: The task you are asked to do it could be create, update or delete
answser create, update or delete

product_name: the name of the product to create, update or delete

price: extract any value related to the product price only if the task is create or update

Format the output as JSON with the following keys:
task
product_name
price

message {message}

{format_instructions}
"""

In [9]:
#define schemas
task_schema = ResponseSchema(name="task",
                             description="The task you are asked to do it could be create, update or delete\
                                answser create, update or delete")
product_name_schema = ResponseSchema(name="product_name",
                                      description="the name of the product to create, update or delete")
price_schema = ResponseSchema(name="price",
                                    description="extract any value related to the product price only if the task is create or update")

response_schemas = [
    task_schema,
    product_name_schema,
    price_schema
]

In [10]:
#set output parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [11]:
#get format instructions
format_instructions = output_parser.get_format_instructions()

In [25]:
prompt_template = ChatPromptTemplate.from_template(prompt)

In [13]:
#initialze agent
assistant = Agent(chat_model, [searchTool], system='You are a smart assistant', checkpointer=memory)

In [14]:
#define thread
thread = {"configurable": {"thread_id": "1"}}

In [15]:
product = """
Create the product cocoa powder with price 2.3
"""

In [16]:
the_message = prompt_template.format_messages(
    message=product,
    format_instructions=format_instructions
)

In [17]:
result = assistant.graph.invoke({'messages':the_message},thread)

In [18]:
result

{'messages': [HumanMessage(content='\nFor the following message extract the following information:\n\ntask: The task you are asked to do it could be create, update or delete\nanswser create, update or delete\n\nproduct_name: the name of the product to create, update or delete\n\nprice: extract any value related to the product price only if the task is create or update\n\nFormat the output as JSON with the following keys:\ntask\nproduct_name\nprice\n\nmessage \nCreate the product cocoa powder with price 2.3\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"task": string  // The task you are asked to do it could be create, update or delete                                answser create, update or delete\n\t"product_name": string  // the name of the product to create, update or delete\n\t"price": string  // extract any value related to the product price only if the task is create or

In [19]:
print(result['messages'][-1].content)

 Sure, I can do that for you. Based on the message "Create the product cocoa powder with price 2.3", the extracted information is as follows:
```json
{
	"task": "create",
	"product_name": "cocoa powder",
	"price": "2.3"
}
```


In [20]:
output = output_parser.parse(result['messages'][-1].content)

In [21]:
output

{'task': 'create', 'product_name': 'cocoa powder', 'price': '2.3'}

In [22]:
type(output)

dict

In [23]:
output.get('task')

'create'